In [ ]:
import os
import sys

# 1. Configure paths and environment variables
base_path = r"c:\Users\dickson\Desktop\1_19_2026\quantai-btc-price-prediction-platform"
python_exe = os.path.join(base_path, "venv", "Scripts", "python.exe")

# 2. Set HADOOP_HOME and PATH
os.environ['HADOOP_HOME'] = os.path.join(base_path, "hadoop")
os.environ['PATH'] = os.path.join(os.environ['HADOOP_HOME'], "bin") + os.pathsep + os.environ['PATH']

# 3. Set Python for Spark workers (Critical for Windows)
os.environ['PYSPARK_PYTHON'] = python_exe
os.environ['PYSPARK_DRIVER_PYTHON'] = python_exe

print(f"HADOOP_HOME: {os.environ['HADOOP_HOME']}")
print(f"PYSPARK_PYTHON: {os.environ['PYSPARK_PYTHON']}")

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("btc_price_prediction") \
    .config(
        "spark.jars.packages",
        "org.postgresql:postgresql:42.7.3"
    ) \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")



your 131072x1 screen size is bogus. expect trouble
26/01/23 14:47:38 WARN Utils: Your hostname, DESKTOP-9V99V1S resolves to a loopback address: 127.0.1.1; using 172.20.61.196 instead (on interface eth0)
26/01/23 14:47:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/khadija/Desktop/BTC-Prediction/quantai-btc-price-prediction-platform/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/khadija/.ivy2/cache
The jars for the packages stored in: /home/khadija/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-47e3f77b-fa11-4eb2-a796-c7a0a8c98dec;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 306ms :: artifacts dl 18ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------

In [2]:
#spark.stop()

## 1. Data Injection (collecte data)

In [3]:
import requests
import os
from dotenv import load_dotenv
load_dotenv() 
def data_collection():

  SYMBOL = 'BTCUSDT'
  INTERVAL = '1m'   # Intervalle d'une minute
  LIMIT = 3000   
  api='https://api.binance.com/api/v3/klines'
  response = requests.get(api, params={
    "symbol": SYMBOL,
    "interval": INTERVAL,
    "limit": LIMIT
  })
  if response.status_code != 200:
        print(f"Erreur {response.status_code}")
        return None
  api_data=response.json()
  columns = [
        "open_time", "open_price", "high_price", "low_price", "close_price", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ]
  psdf= spark.createDataFrame(api_data, columns)
 
  
  return psdf
  
data = data_collection()
data.show(10)

+-------------+--------------+--------------+--------------+--------------+----------+-------------+------------------+----------------+---------------------+----------------------+------+
|    open_time|    open_price|    high_price|     low_price|   close_price|    volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------------+--------------+--------------+--------------+--------------+----------+-------------+------------------+----------------+---------------------+----------------------+------+
|1769116080000|89500.00000000|89548.00000000|89500.00000000|89547.99000000|1.30879000|1769116139999|   117172.79343060|            1263|           0.91681000|        82078.29872180|     0|
|1769116140000|89547.99000000|89568.45000000|89547.99000000|89568.44000000|1.67270000|1769116199999|   149803.18998560|             718|           1.37506000|       123148.68224280|     0|
|1769116200000|89568.44000000|89625.40000000|89568.4400

In [4]:
data.printSchema()

root
 |-- open_time: long (nullable = true)
 |-- open_price: string (nullable = true)
 |-- high_price: string (nullable = true)
 |-- low_price: string (nullable = true)
 |-- close_price: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: string (nullable = true)
 |-- number_of_trades: long (nullable = true)
 |-- taker_buy_base_volume: string (nullable = true)
 |-- taker_buy_quote_volume: string (nullable = true)
 |-- ignore: string (nullable = true)



In [ ]:
print(f"nombre des lignes dans la dataset :",data.count())

nombre des lignes dana la dataset : 1000


#### Conversion des colonnes de type chaîne (string) en valeurs numérique

In [46]:
from pyspark.sql.functions import col
string_cols= ["open_price", "high_price", "low_price", "close_price", "volume", "quote_asset_volume", "number_of_trades", "taker_buy_base_volume", "taker_buy_quote_volume"]
for feature in string_cols:
      data = data.withColumn(feature, col(feature).cast('double'))

data.printSchema()

root
 |-- open_time: long (nullable = true)
 |-- open_price: double (nullable = true)
 |-- high_price: double (nullable = true)
 |-- low_price: double (nullable = true)
 |-- close_price: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: double (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)
 |-- open_time_s: double (nullable = true)
 |-- close_time_s: double (nullable = true)
 |-- open_time_ts: timestamp (nullable = true)
 |-- close_time_ts: timestamp (nullable = true)



In [7]:
data.describe().show()

+-------+--------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+------------------+------------------+---------------------+----------------------+------+
|summary|           open_time|        open_price|        high_price|         low_price|       close_price|           volume|          close_time|quote_asset_volume|  number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------+--------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+------------------+------------------+---------------------+----------------------+------+
|  count|                1000|              1000|              1000|              1000|              1000|             1000|                1000|              1000|              1000|                 1000|                  1000|  1000|
|   mean|       1.76914605E12| 89497.85581000002| 89513.

1. Convertir les colonnes open_time (ms) et close_time (ms) en secondes (s).

In [8]:


data= data.withColumn("open_time_s", col("open_time") / 1000) \
       .withColumn("close_time_s", col("close_time") / 1000)

data.columns
data.select("open_time_s","open_time").show(5)

+------------+-------------+
| open_time_s|    open_time|
+------------+-------------+
|1.76911608E9|1769116080000|
|1.76911614E9|1769116140000|
| 1.7691162E9|1769116200000|
|1.76911626E9|1769116260000|
|1.76911632E9|1769116320000|
+------------+-------------+
only showing top 5 rows



2. Convertir les colonnes open_time_s et close_time_s  en timestamp.

In [9]:
from pyspark.sql.functions import to_timestamp, col

data = data.withColumn("open_time_ts", to_timestamp(col("open_time_s"))) \
       .withColumn("close_time_ts", to_timestamp(col("close_time_s")))
data.printSchema()



root
 |-- open_time: long (nullable = true)
 |-- open_price: double (nullable = true)
 |-- high_price: double (nullable = true)
 |-- low_price: double (nullable = true)
 |-- close_price: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: double (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)
 |-- open_time_s: double (nullable = true)
 |-- close_time_s: double (nullable = true)
 |-- open_time_ts: timestamp (nullable = true)
 |-- close_time_ts: timestamp (nullable = true)



### create le target y = close_price(t+10)

In [10]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Creation une fenêtre ordonnée par temps
window = Window.orderBy("open_time_ts")

# Décaler la colonne 'close_price' de 10 lignes (10 minutes)
data_new = data.withColumn("close_t_plus_10", F.lead("close_price", 10).over(window))
data_new.select("close_price","close_t_plus_10","open_time_ts").show()

+-----------+---------------+-------------------+
|close_price|close_t_plus_10|       open_time_ts|
+-----------+---------------+-------------------+
|   89547.99|       89577.59|2026-01-22 22:08:00|
|   89568.44|       89632.09|2026-01-22 22:09:00|
|   89605.44|       89617.03|2026-01-22 22:10:00|
|    89585.6|       89637.32|2026-01-22 22:11:00|
|    89552.5|       89647.64|2026-01-22 22:12:00|
|    89551.3|       89637.31|2026-01-22 22:13:00|
|   89540.01|       89634.69|2026-01-22 22:14:00|
|    89602.4|       89629.41|2026-01-22 22:15:00|
|   89560.87|       89598.01|2026-01-22 22:16:00|
|   89622.94|       89596.91|2026-01-22 22:17:00|
|   89577.59|       89612.57|2026-01-22 22:18:00|
|   89632.09|       89571.54|2026-01-22 22:19:00|
|   89617.03|       89520.01|2026-01-22 22:20:00|
|   89637.32|        89540.0|2026-01-22 22:21:00|
|   89647.64|       89530.85|2026-01-22 22:22:00|
|   89637.31|       89535.04|2026-01-22 22:23:00|
|   89634.69|       89538.24|2026-01-22 22:24:00|


#### create colonne variation du prix (return) : return = close(t−1) / close(t) − close(t−1)

In [11]:
from pyspark.sql import functions as F
data_new = data_new.withColumn("precedent_close",F.lag("close_price",1).over(window))
data_new = data_new.withColumn('returns',(F.col("close_price")- F.col("precedent_close"))/F.col("precedent_close"))
data_new.select("close_price","precedent_close","returns").show(5)

+-----------+---------------+--------------------+
|close_price|precedent_close|             returns|
+-----------+---------------+--------------------+
|   89547.99|           NULL|                NULL|
|   89568.44|       89547.99|2.283691683084912...|
|   89605.44|       89568.44|4.130919328281256E-4|
|    89585.6|       89605.44|-2.21415128367167...|
|    89552.5|        89585.6|-3.69479023414542...|
+-----------+---------------+--------------------+
only showing top 5 rows



#### Moyennes mobiles (5, 10 minutes)

In [12]:
window5t = window.orderBy("open_time_ts").rowsBetween(-5,-1)
window10t = window.orderBy("open_time_ts").rowsBetween(-10,-1)

data_new = data_new.withColumn("MA_5",F.avg("close_price").over(window5t))
data_new = data_new.withColumn("MA_10",F.avg("close_price").over(window10t))
data_new.select("close_price","open_time_ts","MA_5","MA_10").show(10)


+-----------+-------------------+-----------------+-----------------+
|close_price|       open_time_ts|             MA_5|            MA_10|
+-----------+-------------------+-----------------+-----------------+
|   89547.99|2026-01-22 22:08:00|             NULL|             NULL|
|   89568.44|2026-01-22 22:09:00|         89547.99|         89547.99|
|   89605.44|2026-01-22 22:10:00|        89558.215|        89558.215|
|    89585.6|2026-01-22 22:11:00|89573.95666666667|89573.95666666667|
|    89552.5|2026-01-22 22:12:00|       89576.8675|       89576.8675|
|    89551.3|2026-01-22 22:13:00|89571.99399999999|89571.99399999999|
|   89540.01|2026-01-22 22:14:00|89572.65599999999|        89568.545|
|    89602.4|2026-01-22 22:15:00|         89566.97|89564.46857142857|
|   89560.87|2026-01-22 22:16:00|89566.36200000001|         89569.21|
|   89622.94|2026-01-22 22:17:00|        89561.416|89568.28333333334|
+-----------+-------------------+-----------------+-----------------+
only showing top 10 

#### Volume et Intensité de Trading (Taker Ratio)

In [13]:
data_new = data_new.withColumn("taker_ratio",F.col("taker_buy_base_volume")/F.col("volume"))
data_new.select("open_time_ts", "close_price", "returns",  "MA_5",  "MA_10",  "taker_ratio", "close_t_plus_10").show(10)



+-------------------+-----------+--------------------+-----------------+-----------------+-------------------+---------------+
|       open_time_ts|close_price|             returns|             MA_5|            MA_10|        taker_ratio|close_t_plus_10|
+-------------------+-----------+--------------------+-----------------+-----------------+-------------------+---------------+
|2026-01-22 22:08:00|   89547.99|                NULL|             NULL|             NULL| 0.7005019903880685|       89577.59|
|2026-01-22 22:09:00|   89568.44|2.283691683084912...|         89547.99|         89547.99| 0.8220601422849285|       89632.09|
|2026-01-22 22:10:00|   89605.44|4.130919328281256E-4|        89558.215|        89558.215| 0.8364023878283494|       89617.03|
|2026-01-22 22:11:00|    89585.6|-2.21415128367167...|89573.95666666667|89573.95666666667|0.09052670304739714|       89637.32|
|2026-01-22 22:12:00|    89552.5|-3.69479023414542...|       89576.8675|       89576.8675|0.09863130068351134| 

In [14]:
data_new = data_new.dropna(subset=["returns", "MA_5", "MA_10", "taker_ratio","close_t_plus_10"])
# print(data_new.count())
# data_new.printSchema()

In [15]:
silver_columns = [
        "open_time_ts", "open_price", "high_price", "low_price", "close_price", "volume",
        "returns", "MA_5", "MA_10", "taker_ratio", "close_t_plus_10"
    ]
df_silver = data_new.select(*silver_columns)

In [16]:
df_silver.columns

['open_time_ts',
 'open_price',
 'high_price',
 'low_price',
 'close_price',
 'volume',
 'returns',
 'MA_5',
 'MA_10',
 'taker_ratio',
 'close_t_plus_10']

In [17]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler ,StandardScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor


def training_evaluation_model(data):
 
    features_cols = ["returns", "MA_5", "MA_10", "taker_ratio"]
    
    target = "close_t_plus_10"

    assembler = VectorAssembler(inputCols=features_cols, outputCol="features",handleInvalid="skip")
    window = Window.orderBy("open_time_ts")
    df = data.withColumn("time_index", F.row_number().over(window))

   
    # Séparation séquentielle
    total_count = df.count()
    split_point = int(0.8 * total_count)


    
    train_data = df.filter(F.col("time_index") <= split_point)
    test_data= df.filter(F.col("time_index") > split_point)
    

    
    
    # Transormations
    
    scaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd=True, withMean=True)

    models = [
        ("Linear Regression", LinearRegression(featuresCol="features_scaled", labelCol=target)),
        
        ("Random Forest", RandomForestRegressor(featuresCol="features_scaled", labelCol=target, numTrees=100, maxDepth=10, 
                       seed=42)),
        
        
    ]  
    #  Évaluateurs
    evaluator_r2 = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    evaluator_mae = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")
    evaluator_rmse = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    
    best_rmse = float("inf")
    best_model_name = None
    for model_name, model_algo in models:
        print(f"Entraînement du modèle : {model_name}...")
    
        # Créer pipeline
        pipeline = Pipeline(stages=[assembler, scaler, model_algo])

        # Entraîner modèle
        model_fit = pipeline.fit(train_data)
        # Prédiction
        preds = model_fit.transform(test_data)
        
        # Évaluation
        r2 = evaluator_r2.evaluate(preds)
        mae = evaluator_mae.evaluate(preds)
        rmse = evaluator_rmse.evaluate(preds)
        
        print(f"Modèle: {model_name}")

        print(f"R²: {r2:.4f} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_fit = model_fit
            best_model_name = model_name


    if best_model_fit :
        save_path  = f"saved_model/{best_model_name}_pipeline"
        model_fit.write().overwrite().save(save_path )
        print(f"\n MEILLEUR MODÈLE : {best_model_name} sauvegardé dans '{save_path}' avec un RMSE de {best_rmse:.2f}")
    
    return best_model_fit

In [18]:
training_evaluation_model(df_silver)


Entraînement du modèle : Linear Regression...


Modèle: Linear Regression
R²: 0.4716 | MAE: 59.63 | RMSE: 74.53
Entraînement du modèle : Random Forest...


Modèle: Random Forest
R²: 0.1125 | MAE: 75.60 | RMSE: 96.59



 MEILLEUR MODÈLE : Linear Regression sauvegardé dans 'saved_model/Linear Regression_pipeline' avec un RMSE de 74.53


PipelineModel_a5e5f6103c32

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler ,StandardScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor


def training_evaluation_model(data):
 
    features_cols = ["returns", "MA_5", "MA_10", "taker_ratio"]
    
    target = "close_t_plus_10"

    assembler = VectorAssembler(inputCols=features_cols, outputCol="features",handleInvalid="skip")
    window = Window.orderBy("open_time_ts")
    df = data.withColumn("time_index", F.row_number().over(window))

   
    # Séparation séquentielle
    total_count = df.count()
    split_point = int(0.8 * total_count)


    
    train_data = df.filter(F.col("time_index") <= split_point)
    test_data= df.filter(F.col("time_index") > split_point)
    

    
    
    # Transormations
    
    scaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd=True, withMean=True)

    models = [
        ("Linear Regression", LinearRegression(featuresCol="features_scaled", labelCol=target)),
        
        ("Random Forest", RandomForestRegressor(featuresCol="features_scaled", labelCol=target, numTrees=100, maxDepth=10, 
                       seed=42)),
        ("Linear Regression ElasticNet", LinearRegression(featuresCol="features_scaled", labelCol=target, loss="squaredError", 
        regParam=0.1, 
        elasticNetParam=0.5
         ))
        
    ]  
    #  Évaluateurs
    evaluator_r2 = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    evaluator_mae = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")
    evaluator_rmse = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    
    best_rmse = float("inf")
    best_model_name = None
    for model_name, model_algo in models:
        print(f"Entraînement du modèle : {model_name}...")
    
        # Créer pipeline
        pipeline = Pipeline(stages=[assembler, scaler, model_algo])

        # Entraîner modèle
        model_fit = pipeline.fit(train_data)
        # Prédiction
        preds = model_fit.transform(test_data)
        
        # Évaluation
        r2 = evaluator_r2.evaluate(preds)
        mae = evaluator_mae.evaluate(preds)
        rmse = evaluator_rmse.evaluate(preds)
        
        print(f"Modèle: {model_name}")

        print(f"R²: {r2:.4f} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_fit = model_fit
            best_model_name = model_name


    if best_model_fit :
        save_path  = f"saved_model/{best_model_name}_pipeline"
        model_fit.write().overwrite().save(save_path )
        print(f"\n MEILLEUR MODÈLE : {best_model_name} sauvegardé dans '{save_path}' avec un RMSE de {best_rmse:.2f}")
    
    return best_model_fit

In [20]:
training_evaluation_model(df_silver)

Entraînement du modèle : Linear Regression...


Modèle: Linear Regression
R²: 0.4716 | MAE: 59.63 | RMSE: 74.53
Entraînement du modèle : Random Forest...


Modèle: Random Forest
R²: 0.1125 | MAE: 75.60 | RMSE: 96.59
Entraînement du modèle : Linear Regression ElasticNet...


Modèle: Linear Regression ElasticNet
R²: 0.4721 | MAE: 59.57 | RMSE: 74.49

 MEILLEUR MODÈLE : Linear Regression ElasticNet sauvegardé dans 'saved_model/Linear Regression ElasticNet_pipeline' avec un RMSE de 74.49


PipelineModel_5a9f93e6ca5e

In [28]:
def prepare_ml_data(data, features_cols=None):
    """
    Prépare les données pour l'entraînement ML : création de l'index temporel et split train/test.
    """
    if features_cols is None:
        features_cols = ["returns", "MA_5", "MA_10", "taker_ratio"]

    assembler = VectorAssembler(inputCols=features_cols, outputCol="features", handleInvalid="skip")
    window = Window.orderBy("open_time_ts")
    df = data.withColumn("time_index", F.row_number().over(window))
    
    # Split séquentiel
    total_count = df.count()
    split_point = int(0.8 * total_count)
    
    train_data = df.filter(F.col("time_index") <= split_point)
    test_data = df.filter(F.col("time_index") > split_point)
    
    return train_data, test_data, assembler

In [43]:
import os

def train_evaluate_models(train_data, test_data, assembler, target="close_t_plus_10"):
    scaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd=True, withMean=True)

    models = [
        ("Linear Regression", LinearRegression(featuresCol="features_scaled", labelCol=target)),
        ("Random Forest", RandomForestRegressor(featuresCol="features_scaled", labelCol=target, numTrees=100, maxDepth=10, seed=42)),
        ("Linear Regression ElasticNet", LinearRegression(featuresCol="features_scaled", labelCol=target, regParam=0.1, elasticNetParam=0.5))
    ]
    
    evaluator_r2 = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    evaluator_mae = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")
    evaluator_rmse = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    
    best_rmse = float("inf")
    best_model_fit = None
    best_model_name = None
    
    for model_name, model_algo in models:
        print(f"Entraînement du modèle : {model_name}...")
        pipeline = Pipeline(stages=[assembler, scaler, model_algo])
        model_fit = pipeline.fit(train_data)
        preds = model_fit.transform(test_data)
        
        r2 = evaluator_r2.evaluate(preds)
        mae = evaluator_mae.evaluate(preds)
        rmse = evaluator_rmse.evaluate(preds)
        
        print(f"Modèle: {model_name} | R²: {r2:.4f} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_fit = model_fit
            best_model_name = model_name

    if best_model_fit:
       
        save_path  = f"../saved_model/{best_model_name}_pipeline"
        best_model_fit.write().overwrite().save(save_path)
        print(f"\nMEILLEUR MODÈLE : {best_model_name} sauvegardé dans '{save_path}' avec RMSE {best_rmse:.2f}")
    
    return best_model_fit


In [39]:
train_data, test_data, assembler = prepare_ml_data(df_silver)
train_data.show(5),test_data.show(5)


+-------------------+----------+----------+---------+-----------+-------+--------------------+-----------------+-----------------+-------------------+---------------+----------+
|       open_time_ts|open_price|high_price|low_price|close_price| volume|             returns|             MA_5|            MA_10|        taker_ratio|close_t_plus_10|time_index|
+-------------------+----------+----------+---------+-----------+-------+--------------------+-----------------+-----------------+-------------------+---------------+----------+
|2026-01-22 22:09:00|  89547.99|  89568.45| 89547.99|   89568.44| 1.6727|2.283691683084912...|         89547.99|         89547.99| 0.8220601422849285|       89632.09|         1|
|2026-01-22 22:10:00|  89568.44|   89625.4| 89568.44|   89605.44|5.41915|4.130919328281256E-4|        89558.215|        89558.215| 0.8364023878283494|       89617.03|         2|
|2026-01-22 22:11:00|  89605.43|  89605.44| 89585.59|    89585.6|2.44635|-2.21415128367167...|89573.9566666666

+-------------------+----------+----------+---------+-----------+-------+--------------------+-----------------+-----------------+-------------------+---------------+----------+
|       open_time_ts|open_price|high_price|low_price|close_price| volume|             returns|             MA_5|            MA_10|        taker_ratio|close_t_plus_10|time_index|
+-------------------+----------+----------+---------+-----------+-------+--------------------+-----------------+-----------------+-------------------+---------------+----------+
|2026-01-23 11:20:00|  89092.12|   89110.2| 89092.12|   89110.19|3.00664|2.028237738647030...|         89078.03|        89028.992| 0.8758614267088843|       89093.71|       792|
|2026-01-23 11:21:00|  89110.19|   89110.2| 89094.58|   89094.59|2.35359|-1.75064153718063...|89094.74600000001|89046.61899999999| 0.2415204007494933|       89060.66|       793|
|2026-01-23 11:22:00|  89094.58|  89094.59| 89087.31|   89087.32|1.75814|-8.15986694589371E-5|89099.5340000000

(None, None)

In [44]:
best_model = train_evaluate_models(train_data, test_data, assembler)

Entraînement du modèle : Linear Regression...


Modèle: Linear Regression | R²: 0.4716 | MAE: 59.63 | RMSE: 74.53
Entraînement du modèle : Random Forest...


Modèle: Random Forest | R²: 0.1125 | MAE: 75.60 | RMSE: 96.59
Entraînement du modèle : Linear Regression ElasticNet...


Modèle: Linear Regression ElasticNet | R²: 0.4721 | MAE: 59.57 | RMSE: 74.49



MEILLEUR MODÈLE : Linear Regression ElasticNet sauvegardé dans '../saved_model/Linear Regression ElasticNet_pipeline' avec RMSE 74.49
